In [2]:
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]
# 用户购买记录数据集
datasets = [
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1],
]
import pandas as pd

df = pd.DataFrame(datasets,
                  columns=items,
                  index=users)
print(df)

       Item A  Item B  Item C  Item D  Item E
User1       1       0       1       1       0
User2       1       0       0       1       1
User3       1       0       1       0       0
User4       0       1       0       1       1
User5       1       1       1       0       1


In [3]:
# 直接计算某两项的杰卡德相似系数
from sklearn.metrics import jaccard_score
# 计算Item A 和Item B的相似度
print(jaccard_score(df["Item A"], df["Item C"]))

# 1/6-1  Item A  Item B
# 3/7-3  Item A  Item C

0.75


In [4]:
df.values

array([[1, 0, 1, 1, 0],
       [1, 0, 0, 1, 1],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1],
       [1, 1, 1, 0, 1]], dtype=int64)

In [8]:
# 计算所有的数据两两的杰卡德相似系数
from sklearn.metrics.pairwise import pairwise_distances
# 计算用户间杰卡德相似度----默认计算的是index的杰卡德相似度
print(df.values)
user_similar = 1 - pairwise_distances(df.values, metric="jaccard")
print(user_similar)
user_similar = pd.DataFrame(user_similar, columns=users, index=users)
print("用户之间的两两的杰卡德相似度：")
print(user_similar)

# 计算物品间相似度
item_similar = 1 - pairwise_distances(df.T.values, metric="jaccard")
item_similar = pd.DataFrame(item_similar, columns=items, index=items)
print("物品之间的两两相似度：")
print(item_similar)

[[1 0 1 1 0]
 [1 0 0 1 1]
 [1 0 1 0 0]
 [0 1 0 1 1]
 [1 1 1 0 1]]
[[1.         0.5        0.66666667 0.2        0.4       ]
 [0.5        1.         0.25       0.5        0.4       ]
 [0.66666667 0.25       1.         0.         0.5       ]
 [0.2        0.5        0.         1.         0.4       ]
 [0.4        0.4        0.5        0.4        1.        ]]
用户之间的两两的杰卡德相似度：
          User1  User2     User3  User4  User5
User1  1.000000   0.50  0.666667    0.2    0.4
User2  0.500000   1.00  0.250000    0.5    0.4
User3  0.666667   0.25  1.000000    0.0    0.5
User4  0.200000   0.50  0.000000    1.0    0.4
User5  0.400000   0.40  0.500000    0.4    1.0
物品之间的两两相似度：
        Item A    Item B  Item C  Item D    Item E
Item A    1.00  0.200000    0.75    0.40  0.400000
Item B    0.20  1.000000    0.25    0.25  0.666667
Item C    0.75  0.250000    1.00    0.20  0.200000
Item D    0.40  0.250000    0.20    1.00  0.500000
Item E    0.40  0.666667    0.20    0.50  1.000000


C:\Program Files\Python39\lib\site-packages\sklearn\metrics\pairwise.py:1875: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
C:\Program Files\Python39\lib\site-packages\sklearn\metrics\pairwise.py:1875: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [5]:
print(user_similar.loc['User1'])
user_similar.loc['User1'].drop('User1')

User1    1.000000
User2    0.500000
User3    0.666667
User4    0.200000
User5    0.400000
Name: User1, dtype: float64


User2    0.500000
User3    0.666667
User4    0.200000
User5    0.400000
Name: User1, dtype: float64

In [11]:
from pprint import pprint
topN_users = {}
# 遍历每一行数据
for i in user_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = user_similar.loc[i].drop([i])  #把自己和自己相似度丢弃
    _df_sorted = _df.sort_values(ascending=False)  #降序排列
    print(_df_sorted)

    top2 = list(_df_sorted.index[:2])
    print(top2)
    topN_users[i] = top2

print("Top2相似用户：")
pprint(topN_users)



User3    0.666667
User2    0.500000
User5    0.400000
User4    0.200000
Name: User1, dtype: float64
['User3', 'User2']
User1    0.50
User4    0.50
User5    0.40
User3    0.25
Name: User2, dtype: float64
['User1', 'User4']
User1    0.666667
User5    0.500000
User2    0.250000
User4    0.000000
Name: User3, dtype: float64
['User1', 'User5']
User2    0.5
User5    0.4
User1    0.2
User3    0.0
Name: User4, dtype: float64
['User2', 'User5']
User3    0.5
User1    0.4
User2    0.4
User4    0.4
Name: User5, dtype: float64
['User3', 'User1']
Top2相似用户：
{'User1': ['User3', 'User2'],
 'User2': ['User1', 'User4'],
 'User3': ['User1', 'User5'],
 'User4': ['User2', 'User5'],
 'User5': ['User3', 'User1']}


In [13]:
import numpy as np
rs_results = {}  #存最终结果
# 构建推荐结果
for user, sim_users in topN_users.items():
    rs_result = set()    # 存储推荐结果
    for sim_user in sim_users:
        # 构建初始的推荐结果,要去重 
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0,np.nan).dropna().index))

    # 过滤掉已经购买过的物品
    rs_result -= set(df.loc[user].replace(0,np.nan).dropna().index)
    rs_results[user] = rs_result
print("最终推荐结果：")
pprint(rs_results)

{'Item D', 'Item E', 'Item C', 'Item A'}
*
{'Item D', 'Item E', 'Item C', 'Item A', 'Item B'}
*
{'Item D', 'Item E', 'Item C', 'Item A', 'Item B'}
*
{'Item D', 'Item E', 'Item A', 'Item C', 'Item B'}
*
{'Item D', 'Item C', 'Item A'}
*
最终推荐结果：
{'User1': {'Item E'},
 'User2': {'Item C', 'Item B'},
 'User3': {'Item D', 'Item E', 'Item B'},
 'User4': {'Item A', 'Item C'},
 'User5': {'Item D'}}


In [11]:
#接下来来看皮尔逊
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]
# 用户购买记录数据集
datasets = [
    [5,3,4,4,None],
    [3,1,2,3,3],
    [4,3,4,3,5],
    [3,3,1,5,4],
    [1,5,5,2,1],
]

In [12]:
df = pd.DataFrame(datasets,
                  columns=items,
                  index=users)
print(df)
print("用户之间的两两相似度：")
# 直接计算皮尔逊相关系数
# 默认是按列进行计算，因此如果计算用户间的相似度，当前需要进行转置
user_similar = df.T.corr()
print(user_similar.round(4))

print("物品之间的两两相似度：")  #默认计算的时候colomns的皮尔逊相似度
item_similar = df.corr()
print(item_similar.round(4))

       Item A  Item B  Item C  Item D  Item E
User1       5       3       4       4     NaN
User2       3       1       2       3     3.0
User3       4       3       4       3     5.0
User4       3       3       1       5     4.0
User5       1       5       5       2     1.0
用户之间的两两相似度：
        User1   User2   User3   User4   User5
User1  1.0000  0.8528  0.7071  0.0000 -0.7921
User2  0.8528  1.0000  0.4677  0.4900 -0.9001
User3  0.7071  0.4677  1.0000 -0.1612 -0.4666
User4  0.0000  0.4900 -0.1612  1.0000 -0.6415
User5 -0.7921 -0.9001 -0.4666 -0.6415  1.0000
物品之间的两两相似度：
        Item A  Item B  Item C  Item D  Item E
Item A  1.0000 -0.4767 -0.1231  0.5322  0.9695
Item B -0.4767  1.0000  0.6455 -0.3101 -0.4781
Item C -0.1231  0.6455  1.0000 -0.7206 -0.4276
Item D  0.5322 -0.3101 -0.7206  1.0000  0.5817
Item E  0.9695 -0.4781 -0.4276  0.5817  1.0000


In [22]:
a = ((df.loc['User2']-2.4)*(df.loc['User3']-3.8)).sum()/4/(0.8944*0.8367)
# 皮尔逊如何处理nan
data1=df.drop(index=['User3','User4','User5'],columns='Item E')
print(data1)

       Item A  Item B  Item C  Item D
User1       5       3       4       4
User2       3       1       2       3


In [25]:
b = ((data1.loc['User2']-2.25)*(data1.loc['User1']-4)).sum()/(((data1.loc['User2']-2.25)**2).sum())**0.5/(((data1.loc['User1']-4)**2).sum())**0.5
print(a)
print(b)

0.46769904683468766
0.8528028654224417
Item A    0.48
Item B   -1.12
Item C   -0.32
Item D    0.48
Item E    0.48
Name: User2, dtype: float64


In [27]:
print(((df.loc['User2']-2.4)*(df.loc['User1']-3.8)).sum())

1.88


In [31]:
print(df.loc['User2']-2.4)
print((df.loc['User2']-2.4)*(df.loc['User3']-3.8))
print(((df.loc['User2']-2.4)*(df.loc['User3']-3.8)).sum())
print(df.loc['User1']-3.2)

Item A    0.6
Item B   -1.4
Item C   -0.4
Item D    0.6
Item E    0.6
Name: User2, dtype: float64
Item A    0.12
Item B    1.12
Item C   -0.08
Item D   -0.48
Item E    0.72
dtype: float64
1.4
Item A    1.8
Item B   -0.2
Item C    0.8
Item D    0.8
Item E    NaN
Name: User1, dtype: float64
